In [ ]:
#This notebook contains an implementation to solve the problem min Sum ||A-B_i||_1 + ||2A-11^t||_nucl.
#We solve the problem an the implementation based on "The Augmented Lagrange Multiplier
#method for exact recovery of corrupted low-rank matrices" and some notes of Boyd for ADMM.
#which we link here. https://web.stanford.edu/~boyd/papers/pdf/admm_slides.pdf

#"/although interior point methods normally take very few iterations t converge, they have difficulty in
#handling large matrices because of the complexity of computing the step direction is O(m^6), where m is the dimension of 
#matrix [...] generic interior point solvers are too limited for Robust PCA ".

#Note that the robust PCA is used in many contexts. Ours is the clique problem which deals with nxn symetric matrices
#. However this implementation works for m x n matrices. Our implementation is an extension of the robust pca algorithm.


In [1]:
using JuMP 
using Mosek

In [32]:
#admmRecovery does not call mosek. It uses the prox operation to find xi+1 and Z.
function admmRecovery(observations,lambda,n,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=10000)
    
    #Important! all matrices in the observations are assumed to have 1 in the diagonal.
    #Transformation of the B_i to the C_i where C_i = 2B_i-11^t
    observationsTransformed = transform(observations,n)
    #Initialization (All parameters are based on the paper.)
    k = 0
    N = length(observationsTransformed)
    averObservations=AvergeMatrix(observationsTransformed)
    Y = Array{Any}(N)
    [Y[r]=observationsTransformed[r]/J(observationsTransformed[r],lambda) for r in 1:N]
    Z= averObservations
    Xviejo = Array{Any}(N)
    [Xviejo[r]=zeros(Z) for r in 1:N]
    
    Xnuevo = Array{Any}(N)
    [Xnuevo[r]=zeros(Z) for r in 1:N]
    
    #mu = 1.25/norm(averObservations,2)
    mu=1.1
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true 
        
       
        
        # First solve x^i_k+1 = arg min 1/2||x^i_k-C_i||+mu/2||x^i_k-Z_k+y^i_k||_2^2
 
        [Xnuevo[i] = solveForXprox(observationsTransformed[i],Y[i],Z,mu,n) for i in 1:N]

         # now we solve Z_k+1 = arg min prox_g,rho (1/N sum(x_i_k+1 + 1/rho y_i^k))
        singValDesc = svd(AvergeMatrix(Xnuevo +(1/mu)*Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        Z = singValDesc[1]*SoftThreshold(diagm(singValDesc[2]),lambda/mu)*singValDesc[3]'

        #update Y
        [Y[i] = Y[i]+ mu*(Xnuevo[i]-Z)for i in 1:N]
        
        #check stopping condition
      #  if  firstCriterium(stopCrit1,Z,averObservations,Xnuevo)
       #     println("Done")
        #    break
       # end
        #update mu
        #mu = updateMu(mu,rho,Xviejo,Xnuevo,averObservations,stopCrit2)
         
        #update Xviejo
        Xviejo = Xnuevo
        # update k 
        println("Step $(k)")
        println(norm(Xnuevo[1],2))
        k = k+1
        
        println(mu)
        if k>maxIter
            println("exceeded max number of iteration at solving")
            break
        end    
    end
    return((Z,Xnuevo))
end
        
        
        

admmRecovery (generic function with 5 methods)

In [20]:
#admmRecovery does calls mosek to find xi+1 and Z.

function admmRecovery2(observations,lambda,n,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=10000)
    
    #Important! all matrices in the observations are assumed to have 1 in the diagonal.
    #Transformation of the B_i to the C_i where C_i = 2B_i-11^t
    observationsTransformed = transform(observations,n)
    #Initialization (All parameters are based on the paper.)
    k = 0
    N = length(observationsTransformed)
    averObservations=AvergeMatrix(observationsTransformed)
    Y = Array{Any}(N)
    [Y[r]=observationsTransformed[r]/J(observationsTransformed[r],lambda) for r in 1:N]
    Z= averObservations
    Xviejo = Array{Any}(N)
    [Xviejo[r]=zeros(Z) for r in 1:N]
    
    Xnuevo = Array{Any}(N)
    [Xnuevo[r]=zeros(Z) for r in 1:N]
    
    #mu = 1.25/norm(averObservations,2)
    mu=0.1
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true 
        
       
        
        # First solve x^i_k+1 = arg min ||x^i_k-C_i||+ <yi^k,xi-z^k>   +1/2||x^i_k-Z_k||_2^2
        #for now, we call mosek to solve this
            [Xnuevo[i] = solveForXMosek(observationsTransformed[i],Y[i],Z,mu,n) for i in 1:N]

            

         # now we solve Z_k+1 = arg min prox_g,rho (1/N sum(x_i_k+1 + 1/rho y_i^k))
        singValDesc = svd(AvergeMatrix(Xnuevo) +(1/mu)*AvergeMatrix(Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        Z = singValDesc[1]*SoftThreshold(diagm(singValDesc[2]),lambda/(mu*N))*singValDesc[3]'

        #update Y
        [Y[i] = Y[i]+ mu*(Xnuevo[i]-Z)for i in 1:N]
        
        #check stopping condition
      #  if  firstCriterium(stopCrit1,Z,averObservations,Xnuevo)
       #     println("Done")
        #    break
       # end
        #update mu
        #mu = updateMu(mu,rho,Xviejo,Xnuevo,averObservations,stopCrit2)
        #mu = mu*rho
        #update Xviejo
        Xviejo = Xnuevo
        # update k 
        println("Step $(k)")
        println(norm(Xnuevo[1],2))
        k = k+1
        
        println(mu)
        if k>maxIter
            println("exceeded max number of iteration at solving")
            break
        end    
    end
    return((Z,Xnuevo))
end
        

admmRecovery2 (generic function with 5 methods)

In [21]:
#observations are the observations to compute the function f_i.
#Y is the dual in step k.
#X are the old values of x.
#Z is the objective variable in step k
#mu is the step size.
function solveForXMosek(obs,Y,Z,mu,n)
    m = Model(solver=MosekSolver())
    @variable(m,x[1:n,1:n])
    @variable(m,s[1:n,1:n])
    @constraint(m,-s.<=x-obs)    
    @constraint(m,x-obs.<=s)
    @objective(m,Min,0.5*trace(s'*(ones(n)*ones(n)'))+trace(Y'*(x-Z))+(mu/2)*(trace((x-Z)'*(x-Z))))
     status = solve(m)
    return(getvalue(x))
end
   



solveForX (generic function with 1 method)

In [22]:
function solveForXprox(obs,Y,Z,mu,n)
#    actualizado = SoftThreshold(Z-(1/mu)*Y-obs,1/mu)+obs
    actualizado = SoftThreshold(Z-(1/mu)*Y-obs,1/(2*mu))+obs
    return(actualizado)
end    


solveForX2 (generic function with 1 method)

In [4]:
#Aux functions


#perturbation operator. Perturbs every entry of the matrix W using the function f.
#W is a m x n matriz to perturb
#epsilon is the perturbation
function SoftThreshold(W,perturbation)
    map(W) do x
        perturb(x,perturbation)
    end
end



#Method for perturbing x given epsilon. Used in the softThresholding function.
function perturb(x,epsilon)
    if x>epsilon 
        return x-epsilon
    end
    if x<-epsilon
        return x+epsilon
    else
        return(0)
    end
end



#Method to average matrices

function AvergeMatrix(observations)
    size = length(observations)
    return (1/size)*sum(observations)
end

#J operator
#W is an m x n matriz
#lambda : parameter of the algorithm
function J(W,lambda)
    return max(norm(W,2),(1/lambda)*maximum(abs.(W)))
end

#Function  transform the observed matrices to the desired form. All observed matrices
# have 1 on the diagonal

function transformMatrix(mat,n)
    return(2*mat-ones(n)*ones(n)') 
end

function transform(observations,n)
    return(transformMatrix.(observations,n))
end



function firstCriterium(stopCrit1,objective,D,obs)
    distances = map(x->x-objective,obs)
    if  norm(AvergeMatrix(distances),2)/norm(D,2) < stopCrit1
        return(true)
    end
    return false
end

# Updating functions
#Ek is the value of E computed on the kth step. Ek1 is the value of E computed in the k+1th step.
function updateMu(mu,rho,Xk,Xk1,C,epsilon2)
    
    eval = mu*(norm(AvergeMatrix(Xk-Xk1),2))/norm(AvergeMatrix(C),2)
    #if eval==0
     #   return mu
    #end     
    if eval<epsilon2
        return rho*mu
    end
    return mu
end






updateMu (generic function with 1 method)

<h1>Test<h1>

In [23]:
using JuMP 
using Mosek
using NBInclude
nbinclude("AuxFunctions.ipynb")
nbinclude("GraphLearning.ipynb")

learnRegularizedA (generic function with 1 method)

In [24]:
srand(12536)

MersenneTwister(UInt32[0x000030f8], Base.dSFMT.DSFMT_state(Int32[-642584449, 1073679535, 1179766305, 1073019918, 1125216476, 1073564878, 957469279, 1073717922, 279120579, 1073148466  …  800267814, 1073005159, -793867396, 1073706843, -1618855785, -1651276217, 355150608, -186995734, 382, 0]), [1.84671, 1.03117, 1.32801, 1.17865, 1.00162, 1.74951, 1.09008, 1.68528, 1.27706, 1.77885  …  1.02828, 1.654, 1.48096, 1.06305, 1.85577, 1.48088, 1.994, 1.45974, 1.10458, 1.7444], 382)

In [27]:

grafo20obs5 = Array{Any}(5)
grafo20obs5[1]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
grafo20obs5[2]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
grafo20obs5[3]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
grafo20obs5[4]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
grafo20obs5[5]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
1

1

In [ ]:
#0.28889 s
tic()
recuperaRegular = learnRegularized2A([1/5,1/5,1/5,1/5,1/5],1.6,grafo20obs5,20)
toc()

In [13]:
recuperaRegular[1]

20×20 Array{Float64,2}:
 1.0         1.0          1.0          …  1.06119e-7   4.45827e-8 
 1.0         1.0          1.0             1.18615e-7   6.00183e-8 
 1.0         1.0          1.0             6.09409e-8   1.03277e-10
 1.0         1.0          1.0             5.36996e-8  -2.22209e-10
 1.0         1.0          1.0             5.32627e-8  -3.25964e-9 
 1.0         1.0          1.0          …  7.16925e-8   2.21817e-8 
 1.0         1.0          1.0             5.4247e-8    2.56533e-9 
 1.0         1.0          1.0             1.00549e-7   4.38423e-8 
 1.0         1.0          1.0             5.8494e-8    6.42945e-9 
 1.0         1.0          1.0             4.94299e-8  -7.68553e-9 
 1.0         1.0          1.0          …  6.26549e-8   4.57144e-9 
 1.0         1.0          1.0             9.38817e-8   3.38342e-8 
 6.03361e-8  6.73005e-8   7.16857e-9      1.0          1.0        
 1.16609e-7  1.26125e-7   6.65514e-8      1.0          1.0        
 1.15706e-7  1.26979e-7   6.1432e-8   

In [ ]:
#function admmRecovery2(observations,lambda,n,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=10000)


In [ ]:
#64.3 segundos
tic()
respuestaAdmm = admmRecovery2(grafo20obs5,1.6,20,1.6,1.0e-5,1.0e-5,500)
toc()

In [31]:
bli =(respuestaAdmm[1]+ones(20)*ones(20)')/2

20×20 Array{Float64,2}:
 1.0          1.0          1.0         …   3.83932e-9    4.05544e-9 
 1.0          1.0          1.0            -1.2696e-9    -1.05348e-9 
 1.0          1.0          1.0             3.81038e-9    4.0265e-9  
 1.0          1.0          1.0             4.65253e-9    4.86865e-9 
 1.0          1.0          1.0             2.90721e-9    3.12333e-9 
 1.0          1.0          1.0         …   9.15788e-9    9.374e-9   
 1.0          1.0          1.0            -7.20876e-10  -5.04757e-10
 1.0          1.0          1.0            -6.42067e-10  -4.25948e-10
 1.0          1.0          1.0            -1.22293e-9   -1.00681e-9 
 1.0          1.0          1.0            -6.05223e-10  -3.89105e-10
 1.0          1.0          1.0         …  -3.21656e-10  -1.05537e-10
 1.0          1.0          1.0            -1.09189e-9   -8.75772e-10
 3.99024e-9  -1.11868e-9   3.9613e-9       1.0           1.0        
 3.95991e-9  -1.14902e-9   3.93096e-9      1.0           1.0        
 3.86594e-

In [33]:
#5.67 s
tic()
respuestaAdmmProx = admmRecovery(grafo20obs5,1.6,20,1.6,1.0e-5,1.0e-5,500)
toc()

Step 0
16.04341876213926
1.1
Step 1
19.263690127994423
1.1
Step 2
20.40977347672586
1.1
Step 3
20.383848395697807
1.1
Step 4
20.090263854083712
1.1
Step 5
19.975991048351613
1.1
Step 6
19.974218174328016
1.1
Step 7
19.992319511859183
1.1
Step 8
20.000937865822763
1.1
Step 9
20.00185099740452
1.1
Step 10
20.000743920104377
1.1
Step 11
20.000022872565914
1.1
Step 12
19.99986684216228
1.1
Step 13
19.999926714989552
1.1
Step 14
19.99998878570279
1.1
Step 15
20.000009109536467
1.1
Step 16
20.000007052756438
1.1
Step 17
20.000001783279046
1.1
Step 18
19.9999994589074
1.1
Step 19
19.99999934720115
1.1
Step 20
19.999999775300424
1.1
Step 21
20.00000002056482
1.1
Step 22
20.000000057645167
1.1
Step 23
20.000000025365253
1.1
Step 24
20.000000000971365
1.1
Step 25
19.999999995190556
1.1
Step 26
19.99999999732961
1.1
Step 27
19.9999999996369
1.1
Step 28
20.00000000037226
1.1
Step 29
20.000000000266528
1.1
Step 30
20.000000000058662
1.1
Step 31
19.99999999997435
1.1
Step 32
19.999999999974612
1.1
S

5.672677492

In [34]:
bli =(respuestaAdmmProx[1]+ones(20)*ones(20)')/2

20×20 Array{Float64,2}:
 1.0          1.0          1.0          …  3.33067e-16  1.66533e-16
 1.0          1.0          1.0             2.22045e-16  1.11022e-16
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0          …  1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 1.0          1.0          1.0          …  1.11022e-16  0.0        
 1.0          1.0          1.0             1.11022e-16  0.0        
 5.55112e-17  5.55112e-16  1.11022e-16     1.0          1.0        
 5.55112e-17  5.55112e-16  1.11022e-16     1.0          1.0        
 5.55112e-17  5.55112e-1

<h2>Grafos grandes<h2>

In [35]:
grafo200obs3 = Array{Any}(3)
grafo200obs3[1]=erdosgraph([0.85,0.85],0.1,[100,100],200)+diagm(ones(200))
grafo200obs3[2]=erdosgraph([0.85,0.85],0.1,[100,100],200)+diagm(ones(200))
grafo200obs3[3]=erdosgraph([0.85,0.85],0.1,[100,100],200)+diagm(ones(200))


200×200 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  1.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  0.0  1.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.

In [ ]:
#7 horas
tic()
respuestaAdmmGrande = admmRecovery2(grafo200obs3,1.6,200,1.6,1.0e-5,1.0e-5,200)
toc()

In [12]:
(respuestaAdmmGrande[1]+ones(200)*ones(200)')/2

200×200 Array{Float64,2}:
  0.999991     0.999994     1.00001     …   2.22356e-5   3.47044e-6
  0.999994     0.999998     1.0             1.56104e-5   1.08383e-6
  1.00001      1.0          0.99999        -2.43845e-5  -2.54589e-6
  1.0          1.0          0.999999       -2.01411e-6  -3.71595e-7
  1.00001      6.91444e-6   0.999984       -3.77345e-5  -3.92154e-6
  0.999996     0.999998     1.0         …   7.58836e-6   8.0147e-7 
  1.00001      1.0          0.999983       -2.61186e-5  -2.72654e-6
  1.00001      1.0          0.99999        -2.41558e-5  -3.70371e-6
  1.00001      1.0          0.999994       -1.41018e-5  -1.49836e-6
  0.999987     0.999994     2.19287e-5      3.39515e-5   3.59029e-6
  1.00001      1.0          0.999993    …  -1.7197e-5   -1.77179e-6
  1.00001      1.0          0.99999        -2.24458e-5  -3.4477e-6 
  0.999973    -1.25637e-5   2.95379e-5      4.57386e-5   7.21631e-6
  ⋮                                     ⋱                          
  9.35394e-6   6.57673

In [ ]:
#414 s = 7 min
# sin embargo notamos que desde el step 20 ya no cambia la norma de la matriz asi que volvemos a tratar mas abajo...
tic()
respuestaAdmmProx = admmRecovery(grafo200obs3,1.6,200,1.6,1.0e-5,1.0e-5,200)
toc()

In [37]:
(respuestaAdmmProx[1]+ones(200)*ones(200)')/2

200×200 Array{Float64,2}:
  1.0          1.0          1.0          …  3.4972e-15   3.38618e-15
  1.0          1.0          1.0             2.22045e-16  1.66533e-16
  1.0          1.0          1.0             5.55112e-17  0.0        
  1.0          1.0          1.0             2.22045e-16  1.66533e-16
  1.0          1.0          1.0             2.22045e-16  1.66533e-16
  1.0          1.0          1.0          …  1.66533e-16  5.55112e-17
  1.0          1.0          1.0             1.66533e-16  5.55112e-17
  1.0          1.0          1.0             4.44089e-16  3.33067e-16
  1.0          1.0          1.0             3.88578e-16  2.77556e-16
  1.0          1.0          1.0             4.44089e-16  3.33067e-16
  1.0          1.0          1.0          …  5.55112e-17  0.0        
  1.0          1.0          1.0             2.22045e-16  1.66533e-16
  1.0          1.0          1.0             5.55112e-17  0.0        
  ⋮                                      ⋱                          
  5.5511

In [ ]:
#40 s
tic()
respuestaAdmmProx = admmRecovery(grafo200obs3,1.6,200,1.6,1.0e-5,1.0e-5,20)
toc()

In [39]:
(respuestaAdmmProx[1]+ones(200)*ones(200)')/2

200×200 Array{Float64,2}:
  1.0          1.0          1.0         …  -5.15703e-8  -3.74318e-8
  1.0          1.0          1.0            -6.78639e-8  -5.37254e-8
  1.0          1.0          1.0            -4.37024e-8  -2.95638e-8
  1.0          1.0          1.0            -6.55547e-8  -5.14161e-8
  1.0          1.0          1.0            -8.31886e-8  -6.90501e-8
  1.0          1.0          1.0         …  -5.21917e-8  -3.80531e-8
  1.0          1.0          1.0             4.22659e-8   5.64044e-8
  1.0          1.0          1.0            -7.92237e-8  -6.50851e-8
  1.0          1.0          1.0            -6.19381e-8  -4.77995e-8
  1.0          1.0          1.0            -9.81505e-8  -8.40119e-8
  1.0          1.0          1.0         …  -3.99406e-8  -2.5802e-8 
  1.0          1.0          1.0            -5.12718e-8  -3.71333e-8
  1.0          1.0          1.0            -1.09583e-7  -9.54449e-8
  ⋮                                     ⋱                          
 -2.25652e-8  -3.88588